# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

<h1 align=center><font size = 4>Segmenting and Clustering Neighborhoods in Bangalore City For New Restaurants</font></h1>

 <h2 align=center><font size = 4>Introduction</font></h2>
This Coursera Capstone Project is for IBM Data Science Final Assignment. In this project we have taken a scenario where a person wants to open a new restaurant in the nearby space of an IT office in Bangalore. As opening the new restaurant in these areas is costly affair, it becomes important to analyze the places and existing restaurants around the offices to get a clear idea of the competition there. With the help of Machine Learning, we can not only determine the probable areas but the cuisines can also be analyzed. This project involves gathering location information of the IT offices spaces and using Foursquare APIs to get the location based details. Using the nearby venues we can divide the places in clusters to determine the best options. 

### Importing the Python libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


 <h2 align=center><font size = 4>Data</font></h2>

In this assignment first we have to clean the data obtained from Web page for the Bangalore city and using web scraping method. Then convert addresses into their equivalent latitude and longitude values. Use Foursquare API to explore neighborhoods in Bangalore City. Get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. Use the *k*-means clustering algorithm to complete this task. Finally, use the Folium library to visualize the neighborhoods in Bangalore City and their emerging clusters.This project involves gathering location information of the IT offices spaces and using Foursquare APIs to get the location based details. Using the nearby venues we can divide the places in clusters to determine the best options. 

Based on definition of our problem, factors that will influence our decision are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of offices present in a particular neighborhood 
* Preferable nearest places (Most Valued)

Following data sources will be needed to extract/generate the required information:
* The data is pulled from the site - here, which contains 300 IT company name along with their address. The data also contain PIN code for all the offices. 
* Foursquare API are used to understand the data graphically and get the details of the nearby places

## 1. Scrap data from Wikipedia page into a DataFrame

Get the data set from the URL : https://www.naukri2000.com/careers/it_bangalore.php and then using BeautifulSoup package

In [2]:
dataset = requests.get('https://www.naukri2000.com/careers/it_bangalore.php').text

In [3]:
from bs4 import BeautifulSoup # library to parse HTML and XML documents

In [4]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(dataset, 'html.parser')

### Creating dataframe with the Company_Name and Company_Address

In [5]:
# create three lists to store table data
company_name = []
company_address = []
for row in soup.find_all('table'):
    cells = row.find_all('tr')
for companies in cells:
    company_name.append(companies.contents[2].text)
    company_address.append(companies.contents[4].text)
bangalore_df = pd.DataFrame({"Company_Name": company_name,
                           "Company_Address": company_address})


In [6]:
bangalore_df.head(10)

,Company_Name,Company_Address
0,Name,Address
1,24/7 Customer Pvt Ltd,"Survey No 2/1, 2/2, 2/3, & 5/1, Challaghatta V..."
2,247 Learning Solutions Pvt Ltd,"No 20, Annaswamy Mudaliar Road, Ulsoor Lake, \..."
3,Accenture Services Pvt Ltd,71 Cunningham Road\rBangalore - 560 052
4,Accord Software & Systems Pvt Ltd,"# 37, K.R. Colony, Domlur Layout, \r\nBangalor..."
5,Acme Insurance Services Pvt Ltd,"3rd Floor, Monarch Chambers, \r122, Infantry R..."
6,Adaptec (India) Pvt Ltd,"No:5 , First Floor, \r\nSalarpuria Infinity\r\..."
7,Adea International Pvt Ltd,"No.319/1, Bommanahalli\r\nHosur Main Road\r\nB..."
8,Aditi Technologies Pvt Ltd,224/16 Ramana Maharishi Rd\r\nBangalore 560 080
9,Affiliated Computer Services of India (P) Ltd ...,"Level 2, Creator Block International Tech Park..."


### Extracting the PIN code from address 

In [7]:
# View the data of the dataframe
import re
company_code = []
for addr in bangalore_df['Company_Address']:
   # numbers = re.findall('\d+',addr) 
    nums = addr.split('Bangalore')[-1]
    numbers = re.findall('\d+',nums) 
    #print(addr)
    company_code.append("".join(numbers))

### Appending the PIN Code in the Original dataframe

In [8]:
bangalore_df['Company_Code'] = company_code

### Cleaning Data to remove header

In [9]:
bangalore_df = bangalore_df.loc[1:]

In [10]:
bangalore_df.head()

,Company_Name,Company_Address,Company_Code
1,24/7 Customer Pvt Ltd,"Survey No 2/1, 2/2, 2/3, & 5/1, Challaghatta V...",
2,247 Learning Solutions Pvt Ltd,"No 20, Annaswamy Mudaliar Road, Ulsoor Lake, \...",560042
3,Accenture Services Pvt Ltd,71 Cunningham Road\rBangalore - 560 052,560052
4,Accord Software & Systems Pvt Ltd,"# 37, K.R. Colony, Domlur Layout, \r\nBangalor...",560071
5,Acme Insurance Services Pvt Ltd,"3rd Floor, Monarch Chambers, \r122, Infantry R...",560001


### Correcting the PIN CODE in case of any erroneous code 

In [11]:
bangalore_df['Company_Code'] = bangalore_df['Company_Code'].str[-6:]

### Getting the corresponding Latitude and Longitude details of all the office address using the 'geopy' package 

In [12]:
from geopy.geocoders import Nominatim
lat = []
long = []
for address in bangalore_df['Company_Code']:
    if address:
        geolocator = Nominatim()
        location = geolocator.geocode(address)
        if location:
            #print(location)
            #print((location.latitude, location.longitude))
            lat.append(location.latitude)
            long.append(location.longitude)
        else:
            lat.append("")
            long.append("")
    else:
        lat.append("")
        long.append("")
        

C:\Users\Niteeka Prashant\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


### Appending the Geographical Coordinates to the Original dataframe

In [13]:
bangalore_df['latitude'] = lat
bangalore_df['longitude'] = long

In [14]:
bangalore_df.loc[3:]

,Company_Name,Company_Address,Company_Code,latitude,longitude
3,Accenture Services Pvt Ltd,71 Cunningham Road\rBangalore - 560 052,560052,12.9902,77.596
4,Accord Software & Systems Pvt Ltd,"# 37, K.R. Colony, Domlur Layout, \r\nBangalor...",560071,12.9576,77.6404
5,Acme Insurance Services Pvt Ltd,"3rd Floor, Monarch Chambers, \r122, Infantry R...",560001,-33.0381,137.576
6,Adaptec (India) Pvt Ltd,"No:5 , First Floor, \r\nSalarpuria Infinity\r\...",560029,12.9262,77.5974
7,Adea International Pvt Ltd,"No.319/1, Bommanahalli\r\nHosur Main Road\r\nB...",560068,12.9003,77.6198
8,Aditi Technologies Pvt Ltd,224/16 Ramana Maharishi Rd\r\nBangalore 560 080,560080,13.0001,77.5833
9,Affiliated Computer Services of India (P) Ltd ...,"Level 2, Creator Block International Tech Park...",560066,12.9536,77.7158
10,Ajax.com Pvt Ltd,"#1, 3rd Floor Maruthi Complex, \r\nAbove Food ...",560032,13.0253,77.5984
11,Akamai Technologies India Pvt Ltd,"Salarpuria Ascent\r\n#77, Jyothi Nivas College...",560095,12.9375,77.6179
12,Altair Engineering India Pvt Ltd,"Mercury 2B Block, 5th Floor,\r\nPrestige Tech ...",560078,12.9005,77.5704


As we have got all the required data including the geographicl coordinates of the locations , we can now use the **Foursquare API** to obtain the nearby venue details. First we have to create the URL with all the required details and then use that URL to take the coordinates for each neighborhood and get the venue name and create a dataframe to keep the output

In [15]:
address = 'Bangalore'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore are 12.9791198, 77.5912997.


### Let's create map of Bangalore with all the location details using the folium package

In [16]:
# create map of Bangalore using latitude and longitude values
map_bangalore = folium.Map(location=[latitude, longitude], zoom_start=10)

 # add markers to map
for lat, lng, name1, name2 in zip(bangalore_df['latitude'], bangalore_df['longitude'], bangalore_df['Company_Name'], bangalore_df['Company_Name']):
    if lat and long:
        label = '{}, {}'.format(name1, name2)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_bangalore)  
    
map_bangalore

Create the URL using the CLIENT_ID and CLIENT_SECRET using Foursquare Developers Account

In [49]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20191225'  # Foursquare API version
#'20180605'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [50]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=&client_secret=&v=20191225&ll=12.9791198,77.5912997&radius=500&limit=100'

Using the **Foursquare API** to get the **Venue list** and then create a dataframe using the list for each and every location

In [19]:
venues_list=[]
for name, lat, lng in zip(bangalore_df['Company_Name'],bangalore_df['latitude'],bangalore_df['longitude']):
    print(name)
    if lat and long:      
    # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT)
                
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

24/7 Customer Pvt Ltd 
247 Learning Solutions Pvt Ltd 
Accenture Services Pvt Ltd 
Accord Software & Systems Pvt Ltd 
Acme Insurance Services Pvt Ltd 
Adaptec (India) Pvt Ltd 
Adea International Pvt Ltd 
Aditi Technologies Pvt Ltd 
Affiliated Computer Services of India (P) Ltd (ACS) 
Ajax.com Pvt Ltd 
Akamai Technologies India Pvt Ltd 
Altair Engineering India Pvt Ltd 
Antares Systems Ltd 
ANZ Information Technology Pvt. Ltd. 
Arowana Consulting Ltd 
Artech Infosystems Pvt Ltd 
Ascendas Property Management Services (India) Pvt Ltd 
ASM Technologies Ltd 
Aspect Technology Center (India) Pvt Ltd 
Avaya GlobalConnect Ltd 
Axes Technologies (I) Pvt Ltd 
Aztecsoft Ltd 
BAeHAL Software Limited 
Bahwan CyberTek Pvt Ltd 
Bangalore Softsell Ltd 
Bells Softech Limited 
BEML Technology Division (A div of Bharat Earth Movers Ltd) 
Bharti Telesoft Ltd. 
Blue Chip Computer Consultants Pvt Ltd 
Blue Star Infotech Ltd 
Borland India Pvt Ltd 
Business Process Outsourcing (India) Pvt Ltd 
C1 India Pvt L

Thirdware Solution Ltd. 
Tholons Knowledge Management Pvt Ltd 
Thomson Corporation (International) Pvt Ltd 
ThoughtWorks Technologies India Pvt Ltd 
TIBCO Software India Pvt Ltd 
Timken Engineering and Research India Pvt Ltd 
TPI Advisory Services India Pvt Ltd 
TQM International Pvt Ltd 
Transfleet Global Services Pvt Ltd (TESCO India) 
Trianz Consulting Pvt Ltd 
Tricon Infotech Pvt. Ltd 
Trigent Software Ltd. 
Trilogy E-business Software India Ltd 
Trimentus Technologies Pvt Ltd 
TriVium iCOPE Technologies Pvt Ltd 
TRRS Imaging Ltd 
TRRS Imaging Ltd 
TTK Healthcare Services Pvt Ltd 
U&I Scotty Computers Ltd 
UBICS Technologies Pvt Ltd 
UL India Pvt Ltd 
Unisys Global Services - India (STP Division of Unisys India Pvt Ltd) 
Universal Legal 
USi Internetworking Services Pvt Ltd 
UTL Technologies Ltd. 
Utopia India Pvt Ltd 
Valtech India Technology Solutions Pvt Ltd 
Vee Technologies Pvt Ltd 
Vinciti Networks Pvt Ltd 
Vinpack India pvt. Ltd 
Viteos Capital Market Services Ltd 
vMoksha T

In [20]:
bangalore_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
bangalore_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

#### Let's view the data if we got the details with venue details for all the Neighborhoods

In [21]:
bangalore_venues.head(20)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,247 Learning Solutions Pvt Ltd,12.980207,77.614153,Westside,12.982570,77.610713,Shopping Mall
1,247 Learning Solutions Pvt Ltd,12.980207,77.614153,Mysore Saree Udyog,12.981433,77.610214,Women's Store
2,247 Learning Solutions Pvt Ltd,12.980207,77.614153,Chaipatty ulsoor,12.976061,77.615338,Tea Room
3,247 Learning Solutions Pvt Ltd,12.980207,77.614153,Vashi's House of Jeans,12.981449,77.610308,Clothing Store
4,247 Learning Solutions Pvt Ltd,12.980207,77.614153,Bobby's Punjabi Dhaba,12.983826,77.613955,Indian Restaurant
5,247 Learning Solutions Pvt Ltd,12.980207,77.614153,Cafe Coffee Day Ulsoor Lake,12.979498,77.618151,Café
6,247 Learning Solutions Pvt Ltd,12.980207,77.614153,Sreeraj Lassi Bar,12.982748,77.610739,Juice Bar
7,247 Learning Solutions Pvt Ltd,12.980207,77.614153,Anand Sweets,12.981667,77.609790,Candy Store
8,247 Learning Solutions Pvt Ltd,12.980207,77.614153,Baskin-Robbins,12.975867,77.614559,Ice Cream Shop
9,247 Learning Solutions Pvt Ltd,12.980207,77.614153,Sri Krishna Diamonds and Jewellery,12.981445,77.610212,Jewelry Store


In [22]:
print(bangalore_venues.shape)

(6949, 7)


#### Taking the count of each Comapany Address to analyze how many the Venue are obtained for each Company

In [23]:
bangalore_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
247 Learning Solutions Pvt Ltd,12,12,12,12,12,12
ANZ Information Technology Pvt. Ltd.,25,25,25,25,25,25
ASM Technologies Ltd,5,5,5,5,5,5
Accenture Services Pvt Ltd,35,35,35,35,35,35
Accord Software & Systems Pvt Ltd,25,25,25,25,25,25
Acme Insurance Services Pvt Ltd,6,6,6,6,6,6
Adaptec (India) Pvt Ltd,8,8,8,8,8,8
Adea International Pvt Ltd,4,4,4,4,4,4
Aditi Technologies Pvt Ltd,7,7,7,7,7,7


In [24]:
print('There are {} uniques categories.'.format(len(bangalore_venues['Venue Category'].unique())))

There are 143 uniques categories.


### Onehot coding of the venues which is a categorical data to convert it to format required by ML Algorithm.

In [25]:
# one hot encoding
bangalore_onehot = pd.get_dummies(bangalore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bangalore_onehot['Neighborhood'] = bangalore_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bangalore_onehot.columns[-1]] + list(bangalore_onehot.columns[:-1])
bangalore_onehot = bangalore_onehot[fixed_columns]

bangalore_onehot.head()

,Neighborhood,ATM,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Bistro,Boarding House,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Campground,Candy Store,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,Concert Hall,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Design Studio,Dessert Shop,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Furniture / Home Store,Gaming Cafe,Gastropub,General Entertainment,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,Hookah Bar,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Mughlai Restaurant,Multiplex,Music Venue,Office,Optical Shop,Park,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Platform,Pool Hall,Pub,Resort,Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Tea Room,Tennis Court,Tex-Mex Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store
0,247 Learning Solutions Pvt Ltd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,247 Learning Solutions Pvt Ltd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,247 Learning Solutions Pvt Ltd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,247 Learning Solutions Pvt Ltd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,247 Learning Solutions Pvt Ltd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
bangalore_onehot.shape

(6949, 144)

#### Grouping the data and taking the mean of all the Venue so that dataframe size is reduced and relevant data is gathered

In [27]:
bangalore_grouped = bangalore_onehot.groupby('Neighborhood').mean().reset_index()
bangalore_grouped

,Neighborhood,ATM,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Bistro,Boarding House,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Campground,Candy Store,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,Concert Hall,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Design Studio,Dessert Shop,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Furniture / Home Store,Gaming Cafe,Gastropub,General Entertainment,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,Hookah Bar,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Mughlai Restaurant,Multiplex,Music Venue,Office,Optical Shop,Park,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Platform,Pool Hall,Pub,Resort,Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Tea Room,Tennis Court,Tex-Mex Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store
0,247 Learning Solutions Pvt Ltd,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.0,0.083333,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.083333,0.000000,0.00,0.00,0.000000,0.000000,0.083333,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.083333,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333
1,ANZ Information Technology Pvt. Ltd.,0.0,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.040000,0.000000,0.040000,0.000000,0.0,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.040000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.04,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.04,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.000000,0.160000,0.000000,0.000000,0.000000,0.000000,0.240000,0.000000,0.00,0.

In [28]:
bangalore_grouped.shape

(262, 144)

### Analyzing the top 5 Venues for each Neighborhood based on the mean/frequency obtained above

In [29]:
num_top_venues = 5

for hood in bangalore_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bangalore_grouped[bangalore_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----247 Learning Solutions Pvt Ltd ----
                     venue  freq
0            Women's Store  0.08
1                 Tea Room  0.08
2            Shopping Mall  0.08
3                     Café  0.08
4  South Indian Restaurant  0.08


----ANZ Information Technology Pvt. Ltd. ----
               venue  freq
0  Indian Restaurant  0.24
1              Hotel  0.16
2        Pizza Place  0.08
3           Tea Room  0.04
4               Food  0.04


----ASM Technologies Ltd ----
               venue  freq
0  College Cafeteria   0.4
1               Park   0.2
2              Field   0.2
3        Snack Place   0.2
4          Multiplex   0.0


----Accenture Services Pvt Ltd ----
                venue  freq
0   Indian Restaurant  0.26
1         Coffee Shop  0.09
2               Hotel  0.06
3  Italian Restaurant  0.06
4         Pizza Place  0.06


----Accord Software & Systems Pvt Ltd ----
               venue  freq
0  Indian Restaurant  0.24
1              Hotel  0.16
2        Pizza Place  0.08

4            Food Court  0.12


----Centre for Development of Advanced Computing (C - DAC) ----
                venue  freq
0   Indian Restaurant  0.29
1  Chinese Restaurant  0.29
2    Department Store  0.14
3   Andhra Restaurant  0.14
4  Athletics & Sports  0.14


----Citigroup Information Technology Operations and Solutions Ltd ----
                    venue  freq
0       Food & Drink Shop  0.17
1  Thrift / Vintage Store  0.17
2           Grocery Store  0.17
3                   Beach  0.17
4           Bowling Alley  0.17


----Citrix Systems India Pvt Ltd ----
                venue  freq
0   Indian Restaurant  0.26
1         Coffee Shop  0.09
2               Hotel  0.06
3  Italian Restaurant  0.06
4         Pizza Place  0.06


----City Info Services Pvt Ltd ----
                    venue  freq
0       Food & Drink Shop  0.17
1  Thrift / Vintage Store  0.17
2           Grocery Store  0.17
3                   Beach  0.17
4           Bowling Alley  0.17


----Comat Technologies (P) Ltd.

                venue  freq
0   Indian Restaurant  0.22
1  Chinese Restaurant  0.08
2                Café  0.06
3        Dessert Shop  0.04
4          Restaurant  0.03


----IDEB Construction Projects (P) Ltd ----
               venue  freq
0  Indian Restaurant  0.36
1      Shopping Mall  0.07
2     Ice Cream Shop  0.07
3          Bookstore  0.07
4        Pizza Place  0.07


----ITTI Limited ----
                  venue  freq
0     Indian Restaurant  0.33
1  Fast Food Restaurant  0.11
2           Coffee Shop  0.06
3           Pizza Place  0.06
4        Farmers Market  0.06


----ITTI Pvt. Ltd ----
                  venue  freq
0     Indian Restaurant  0.33
1  Fast Food Restaurant  0.11
2           Coffee Shop  0.06
3           Pizza Place  0.06
4        Farmers Market  0.06


----IVL India Pvt Ltd ----
                venue  freq
0   Indian Restaurant  0.26
1         Coffee Shop  0.09
2               Hotel  0.06
3  Italian Restaurant  0.06
4         Pizza Place  0.06


----Impact Solut

               venue  freq
0  Indian Restaurant  0.24
1              Hotel  0.16
2        Pizza Place  0.08
3           Tea Room  0.04
4               Food  0.04


----Newgen Software Technologies Ltd ----
               venue  freq
0               Café  0.10
1  Indian Restaurant  0.08
2              Hotel  0.06
3         Donut Shop  0.05
4                Pub  0.05


----Next Link (P) Ltd ----
                    venue  freq
0       Food & Drink Shop  0.17
1  Thrift / Vintage Store  0.17
2           Grocery Store  0.17
3                   Beach  0.17
4           Bowling Alley  0.17


----Nich - In Software Solutions Pvt. Ltd. ----
                  venue  freq
0  Fast Food Restaurant  0.18
1     Indian Restaurant  0.14
2           Pizza Place  0.14
3         Shopping Mall  0.09
4        Ice Cream Shop  0.09


----Nirvana Business Solutions Pvt Ltd ----
               venue  freq
0  Indian Restaurant  0.36
1      Shopping Mall  0.07
2     Ice Cream Shop  0.07
3          Bookstore  0.07


4        Gourmet Shop   0.1


----STAG Software Pvt Ltd ----
               venue  freq
0  Indian Restaurant  0.24
1              Hotel  0.16
2        Pizza Place  0.08
3           Tea Room  0.04
4               Food  0.04


----SVB India Advisors Pvt Ltd ----
               venue  freq
0               Café  0.10
1  Indian Restaurant  0.08
2              Hotel  0.06
3         Donut Shop  0.05
4                Pub  0.05


----Saastha Infotech Pvt Ltd ----
                 venue  freq
0        Historic Site  0.25
1          Bus Station  0.25
2     Football Stadium  0.25
3  Indie Movie Theater  0.25
4            Multiplex  0.00


----Sampoorna Matchjobs.com Pvt Ltd ----
                venue  freq
0   Indian Restaurant  0.29
1  Chinese Restaurant  0.29
2    Department Store  0.14
3   Andhra Restaurant  0.14
4  Athletics & Sports  0.14


----Samsung Electronics India Software Operations (SISO) ----
               venue  freq
0  Indian Restaurant   0.3
1        Pizza Place   0.2
2     Shop 

                venue  freq
0     Motorcycle Shop  0.25
1      Clothing Store  0.25
2   Indian Restaurant  0.25
3           Pool Hall  0.25
4  Miscellaneous Shop  0.00


----TRRS Imaging Ltd ----
                venue  freq
0   Indian Restaurant  0.18
1  Chinese Restaurant  0.18
2              Bakery  0.09
3   Andhra Restaurant  0.09
4         Pizza Place  0.09


----Talisma Corporation Pvt Ltd ----
               venue  freq
0  Indian Restaurant  0.29
1              Hotel  0.14
2               Café  0.14
3  Andhra Restaurant  0.14
4     Ice Cream Shop  0.14


----Tally Solutions Pvt Ltd ----
                venue  freq
0   Indian Restaurant  0.21
1                Café  0.16
2      Ice Cream Shop  0.11
3  Chinese Restaurant  0.11
4        Burger Joint  0.05


----Tandon Information Solutions Pvt. Ltd. ----
                           venue  freq
0  Vegetarian / Vegan Restaurant  0.33
1                           Café  0.17
2              Indian Restaurant  0.17
3            Japanese Rest

                venue  freq
0   Indian Restaurant  0.22
1  Chinese Restaurant  0.08
2                Café  0.06
3        Dessert Shop  0.04
4          Restaurant  0.03


----Viteos Capital Market Services Ltd ----
                   venue  freq
0                  Hotel  0.21
1                   Café  0.21
2      Indian Restaurant  0.14
3            Bus Station  0.07
4  Hyderabadi Restaurant  0.07


----WYSE Technology Sales & Marketing India Pvt Ltd ----
               venue  freq
0  Indian Restaurant  0.36
1      Shopping Mall  0.07
2     Ice Cream Shop  0.07
3          Bookstore  0.07
4        Pizza Place  0.07


----Wind River Systems, Inc ----
                    venue  freq
0       Food & Drink Shop  0.17
1  Thrift / Vintage Store  0.17
2           Grocery Store  0.17
3                   Beach  0.17
4           Bowling Alley  0.17


----Winfoware Technologies Pvt Ltd ----
                  venue  freq
0  Fast Food Restaurant  0.25
1     Indian Restaurant  0.17
2           Snack Pl

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Obtaining the Top 10 Venues of each of the Company Name

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bangalore_grouped['Neighborhood']

for ind in np.arange(bangalore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bangalore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,247 Learning Solutions Pvt Ltd,Women's Store,Ice Cream Shop,Multiplex,Candy Store,Café,Juice Bar,Jewelry Store,South Indian Restaurant,Indian Restaurant,Shopping Mall
1,ANZ Information Technology Pvt. Ltd.,Indian Restaurant,Hotel,Pizza Place,Coffee Shop,Pub,Bus Station,Pharmacy,Irish Pub,Café,Food
2,ASM Technologies Ltd,College Cafeteria,Snack Place,Park,Field,Department Store,Eastern European Restaurant,Donut Shop,Diner,Dessert Shop,Design Studio
3,Accenture Services Pvt Ltd,Indian Restaurant,Coffee Shop,Hotel,Italian Restaurant,Pizza Place,Middle Eastern Restaurant,Dessert Shop,Salad Place,Sandwich Place,Fast Food Restaurant
4,Accord Software & Systems Pvt Ltd,Indian Restaurant,Hotel,Pizza Place,Coffee Shop,Pub,Bus Station,Pharmacy,Irish Pub,Café,Food


 <h2 align=center><font size = 4>Clustering</font></h2>

Now we have got the required data to start with the Machine Learning Algorithm and get the cluster details. Initially we will be starting with 5 Clusters to segment the data.

### Initializing the K-Means clustering algorithm with 5 clusters

In [32]:
# set number of clusters
kclusters = 5

bangalore_grouped_clustering = bangalore_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bangalore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 2, 1, 1, 4, 1, 3, 1, 1])

In [33]:
bangalore_df.rename(columns={'Company_Name': 'Neighborhood'},inplace=True) 

In [35]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,247 Learning Solutions Pvt Ltd,Women's Store,Ice Cream Shop,Multiplex,Candy Store,Café,Juice Bar,Jewelry Store,South Indian Restaurant,Indian Restaurant,Shopping Mall
1,ANZ Information Technology Pvt. Ltd.,Indian Restaurant,Hotel,Pizza Place,Coffee Shop,Pub,Bus Station,Pharmacy,Irish Pub,Café,Food
2,ASM Technologies Ltd,College Cafeteria,Snack Place,Park,Field,Department Store,Eastern European Restaurant,Donut Shop,Diner,Dessert Shop,Design Studio
3,Accenture Services Pvt Ltd,Indian Restaurant,Coffee Shop,Hotel,Italian Restaurant,Pizza Place,Middle Eastern Restaurant,Dessert Shop,Salad Place,Sandwich Place,Fast Food Restaurant
4,Accord Software & Systems Pvt Ltd,Indian Restaurant,Hotel,Pizza Place,Coffee Shop,Pub,Bus Station,Pharmacy,Irish Pub,Café,Food


#### Merging the Original and the Top 10 venues and the ClusterLabel obtained by the model for each Neighborhood

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)
bangalore_merged = bangalore_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bangalore_merged = bangalore_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bangalore_merged.head() # check the last columns!

,Neighborhood,Company_Address,Company_Code,latitude,longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,24/7 Customer Pvt Ltd,"Survey No 2/1, 2/2, 2/3, & 5/1, Challaghatta V...",,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,247 Learning Solutions Pvt Ltd,"No 20, Annaswamy Mudaliar Road, Ulsoor Lake, \...",560042,12.9802,77.6142,2.0,Women's Store,Ice Cream Shop,Multiplex,Candy Store,Café,Juice Bar,Jewelry Store,South Indian Restaurant,Indian Restaurant,Shopping Mall
3,Accenture Services Pvt Ltd,71 Cunningham Road\rBangalore - 560 052,560052,12.9902,77.596,1.0,Indian Restaurant,Coffee Shop,Hotel,Italian Restaurant,Pizza Place,Middle Eastern Restaurant,Dessert Shop,Salad Place,Sandwich Place,Fast Food Restaurant
4,Accord Software & Systems Pvt Ltd,"# 37, K.R. Colony, Domlur Layout, \r\nBangalor...",560071,12.9576,77.6404,1.0,Indian Restaurant,Hotel,Pizza Place,Coffee Shop,Pub,Bus Station,Pharmacy,Irish Pub,Café,Food
5,Acme Insurance Services Pvt Ltd,"3rd Floor, Monarch Chambers, \r122, Infantry R...",560001,-33.0381,137.576,4.0,Café,Grocery Store,Bowling Alley,Thrift / Vintage Store,Beach,Food & Drink Shop,Design Studio,Eastern European Restaurant,Donut Shop,Diner


In [37]:
bangalore_merged['ClusterLabels'].dropna(inplace=True)

In [38]:
bangalore_merged = bangalore_merged.dropna()
bangalore_merged = bangalore_merged.reset_index(drop=True)

In [39]:
bangalore_merged['ClusterLabels'] = bangalore_merged['ClusterLabels'].astype(int)

In [40]:
bangalore_merged.head()

,Neighborhood,Company_Address,Company_Code,latitude,longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,247 Learning Solutions Pvt Ltd,"No 20, Annaswamy Mudaliar Road, Ulsoor Lake, \...",560042,12.9802,77.6142,2,Women's Store,Ice Cream Shop,Multiplex,Candy Store,Café,Juice Bar,Jewelry Store,South Indian Restaurant,Indian Restaurant,Shopping Mall
1,Accenture Services Pvt Ltd,71 Cunningham Road\rBangalore - 560 052,560052,12.9902,77.596,1,Indian Restaurant,Coffee Shop,Hotel,Italian Restaurant,Pizza Place,Middle Eastern Restaurant,Dessert Shop,Salad Place,Sandwich Place,Fast Food Restaurant
2,Accord Software & Systems Pvt Ltd,"# 37, K.R. Colony, Domlur Layout, \r\nBangalor...",560071,12.9576,77.6404,1,Indian Restaurant,Hotel,Pizza Place,Coffee Shop,Pub,Bus Station,Pharmacy,Irish Pub,Café,Food
3,Acme Insurance Services Pvt Ltd,"3rd Floor, Monarch Chambers, \r122, Infantry R...",560001,-33.0381,137.576,4,Café,Grocery Store,Bowling Alley,Thrift / Vintage Store,Beach,Food & Drink Shop,Design Studio,Eastern European Restaurant,Donut Shop,Diner
4,Adaptec (India) Pvt Ltd,"No:5 , First Floor, \r\nSalarpuria Infinity\r\...",560029,12.9262,77.5974,1,Indian Restaurant,Asian Restaurant,Restaurant,Food Court,Fast Food Restaurant,Concert Hall,Cosmetics Shop,College Cafeteria,Cupcake Shop,Dance Studio


 <h2 align=center><font size = 4>Result</font></h2>

As we can see that we got the Clusters Labeled for all the Neighborhoods, it shows all the details with most common venues 

### Once again visualizing the Map of the bangalore but this time with segmentation and clusters 

In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bangalore_merged['latitude'], bangalore_merged['longitude'], bangalore_merged['Neighborhood'], bangalore_merged['ClusterLabels']):
    if lat and long:
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Analyzing the Cluster Data with different categorization

In [43]:
bangalore_merged.loc[bangalore_merged['ClusterLabels'] == 0, bangalore_merged.columns[[1] + list(range(5, bangalore_merged.shape[1]))]]

,Company_Address,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,"887, \r\n14 A Cross\r\nIndira Nagar 2nd Stage\...",0,Chinese Restaurant,Indian Restaurant,Department Store,Andhra Restaurant,Athletics & Sports,Design Studio,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop
28,"2nd Floor, Salarpuria House\r\nNo. 496, CMH Ro...",0,Chinese Restaurant,Indian Restaurant,Department Store,Andhra Restaurant,Athletics & Sports,Design Studio,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop
39,C-DAC Knowledge Park\r\nOpp. HAL Aeroengine Di...,0,Chinese Restaurant,Indian Restaurant,Department Store,Andhra Restaurant,Athletics & Sports,Design Studio,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop
77,"841/1, Binnamangala, 100FT Road, Indiranagar\r...",0,Chinese Restaurant,Indian Restaurant,Department Store,Andhra Restaurant,Athletics & Sports,Design Studio,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop
111,"44121/B, 19th A Main, 6th Cross, HAL II Stage ...",0,Chinese Restaurant,Indian Restaurant,Department Store,Andhra Restaurant,Athletics & Sports,Design Studio,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop
159,"787, 1st Cross, 12th Main, \r\nHAL 2nd Stage, ...",0,Chinese Restaurant,Indian Restaurant,Department Store,Andhra Restaurant,Athletics & Sports,Design Studio,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop
171,"Sampoorna Computer People\r\n#482, 2nd Floor, ...",0,Chinese Restaurant,Indian Restaurant,Department Store,Andhra Restaurant,Athletics & Sports,Design Studio,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop
188,"#803, 7th Main, 1st Cross\r\nH.A.L. 2nd Stage\...",0,Chinese Restaurant,Indian Restaurant,Department Store,Andhra Restaurant,Athletics & Sports,Design Studio,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop
206,"1st Floor, Unit No. 3, No. 34, 11th Cross, \nI...",0,Chinese Restaurant,Indian Restaurant,Department Store,Andhra Restaurant,Athletics & Sports,Design Studio,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop
207,"Sanata Clara, 3rd Floor, 18th Main, \r\n1st Cr...",0,Chinese Restaurant,Indian Restaurant,Department Store,Andhra Restaurant,Athletics & Sports,Design Studio,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop


In [44]:
bangalore_merged.loc[bangalore_merged['ClusterLabels'] == 1, bangalore_merged.columns[[1] + list(range(5, bangalore_merged.shape[1]))]]

,Company_Address,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,71 Cunningham Road\rBangalore - 560 052,1,Indian Restaurant,Coffee Shop,Hotel,Italian Restaurant,Pizza Place,Middle Eastern Restaurant,Dessert Shop,Salad Place,Sandwich Place,Fast Food Restaurant
2,"# 37, K.R. Colony, Domlur Layout, \r\nBangalor...",1,Indian Restaurant,Hotel,Pizza Place,Coffee Shop,Pub,Bus Station,Pharmacy,Irish Pub,Café,Food
4,"No:5 , First Floor, \r\nSalarpuria Infinity\r\...",1,Indian Restaurant,Asian Restaurant,Restaurant,Food Court,Fast Food Restaurant,Concert Hall,Cosmetics Shop,College Cafeteria,Cupcake Shop,Dance Studio
6,224/16 Ramana Maharishi Rd\r\nBangalore 560 080,1,Indian Restaurant,Andhra Restaurant,Hotel,Dance Studio,Ice Cream Shop,Café,Women's Store,Design Studio,Electronics Store,Eastern European Restaurant
7,"Level 2, Creator Block International Tech Park...",1,Indian Restaurant,Chinese Restaurant,Ice Cream Shop,Café,Pizza Place,Sandwich Place,Bookstore,Fast Food Restaurant,BBQ Joint,Shopping Mall
9,"Salarpuria Ascent\r\n#77, Jyothi Nivas College...",1,Indian Restaurant,Chinese Restaurant,Café,Dessert Shop,Pub,Snack Place,Restaurant,Lounge,Asian Restaurant,Electronics Store
11,"#24, 1st Floor, Sudha Complex, \r\n3rd Stage, ...",1,Fast Food Restaurant,Indian Restaurant,Sandwich Place,Juice Bar,Gastropub,Snack Place,Asian Restaurant,Burger Joint,Ice Cream Shop,Dessert Shop
12,Cherry Hills\r\nEmbassy Golf Links Business Pa...,1,Indian Restaurant,Hotel,Pizza Place,Coffee Shop,Pub,Bus Station,Pharmacy,Irish Pub,Café,Food
14,"#50, HIG, KHB Colony, \r2nd Floor, 80 Feet Roa...",1,Indian Restaurant,Chinese Restaurant,Café,Dessert Shop,Pub,Snack Place,Restaurant,Lounge,Asian Restaurant,Electronics Store
15,"3rd Floor, Discoverer Building\rInternational ...",1,Indian Restaurant,Chinese Restaurant,Ice Cream Shop,Café,Pizza Place,Sandwich Place,Bookstore,Fast Food Restaurant,BBQ Joint,Shopping Mall


In [45]:
bangalore_merged.loc[bangalore_merged['ClusterLabels'] == 2, bangalore_merged.columns[[1] + list(range(5, bangalore_merged.shape[1]))]]

,Company_Address,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"No 20, Annaswamy Mudaliar Road, Ulsoor Lake, \...",2,Women's Store,Ice Cream Shop,Multiplex,Candy Store,Café,Juice Bar,Jewelry Store,South Indian Restaurant,Indian Restaurant,Shopping Mall
8,"#1, 3rd Floor Maruthi Complex, \r\nAbove Food ...",2,Fast Food Restaurant,Gym,Resort,Park,Donut Shop,Diner,Dessert Shop,Design Studio,Department Store,Dance Studio
10,"Mercury 2B Block, 5th Floor,\r\nPrestige Tech ...",2,Health & Beauty Service,Motorcycle Shop,Spa,Metro Station,College Auditorium,College Cafeteria,Concert Hall,Cosmetics Shop,Cupcake Shop,Farmers Market
16,"80/2 Lusanne Court, \r\nRichmond Road, \r\nBan...",2,College Cafeteria,Snack Place,Park,Field,Department Store,Eastern European Restaurant,Donut Shop,Diner,Dessert Shop,Design Studio
23,"No.17, Second Floor, 100 ft. Ring Road, \r\n3r...",2,Fast Food Restaurant,Scenic Lookout,Ice Cream Shop,Department Store,Eastern European Restaurant,Donut Shop,Diner,Dessert Shop,Design Studio,Dance Studio
24,"# 91, HRS Chambers\r\nLevel - 1, Richmond Road...",2,Café,Indian Restaurant,Hotel,Pub,Bakery,Donut Shop,Clothing Store,Bar,Sandwich Place,Electronics Store
25,BEML Technology Division\r\nBEML SOUDHA\r\n23/...,2,Vegetarian / Vegan Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Café,Indian Restaurant,Women's Store,Department Store,Eastern European Restaurant,Donut Shop,Diner
26,"Maruti Towers, \r138, Airport Road, \rBangalor...",2,Indian Restaurant,Pub,Lounge,Restaurant,Vietnamese Restaurant,Tea Room,Italian Restaurant,Café,Brewery,Diner
29,"211, Ranka Court, 2nd Block, \r18 Cambridge La...",2,Indian Restaurant,Pub,Lounge,Restaurant,Vietnamese Restaurant,Tea Room,Italian Restaurant,Café,Brewery,Diner
32,"Anchorage, \r\n# 100/1, Richmond Road, \r\nBan...",2,Café,Indian Restaurant,Hotel,Pub,Bakery,Donut Shop,Clothing Store,Bar,Sandwich Place,Electronics Store


In [46]:
bangalore_merged.loc[bangalore_merged['ClusterLabels'] == 3, bangalore_merged.columns[[1] + list(range(5, bangalore_merged.shape[1]))]]

,Company_Address,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,"No.319/1, Bommanahalli\r\nHosur Main Road\r\nB...",3,Gym / Fitness Center,Basketball Court,South Indian Restaurant,Indian Restaurant,Concert Hall,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Design Studio
27,"20-21, Pattalamma Layout, Singasandra, \rHosur...",3,Gym / Fitness Center,Basketball Court,South Indian Restaurant,Indian Restaurant,Concert Hall,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Design Studio
53,"""Shanthi Shree"", \r\n#17/1, Hosur Road, \r\nBa...",3,Gym / Fitness Center,Basketball Court,South Indian Restaurant,Indian Restaurant,Concert Hall,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Design Studio
83,"HTMT House No. 614, Vajpayee Nagar, \r\nBomman...",3,Gym / Fitness Center,Basketball Court,South Indian Restaurant,Indian Restaurant,Concert Hall,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Design Studio
135,"49/1 &49/3, Garvebhavipalya, 7th Mile, Hosur R...",3,Gym / Fitness Center,Basketball Court,South Indian Restaurant,Indian Restaurant,Concert Hall,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Design Studio
136,"49/1 & 49/3 Garvebhavi Palya\r\n7th Mile, Hosu...",3,Gym / Fitness Center,Basketball Court,South Indian Restaurant,Indian Restaurant,Concert Hall,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Design Studio
224,"No. 10, 3rd Floor, \r\n100 Feet Ring Road, \r\...",3,Gym / Fitness Center,Basketball Court,South Indian Restaurant,Indian Restaurant,Concert Hall,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Design Studio


In [47]:
bangalore_merged.loc[bangalore_merged['ClusterLabels'] == 4, bangalore_merged.columns[[1] + list(range(5, bangalore_merged.shape[1]))]]

,Company_Address,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"3rd Floor, Monarch Chambers, \r122, Infantry R...",4,Café,Grocery Store,Bowling Alley,Thrift / Vintage Store,Beach,Food & Drink Shop,Design Studio,Eastern European Restaurant,Donut Shop,Diner
22,"#30, Prestige Meridian II, \rUnit 701 & 702, \...",4,Café,Grocery Store,Bowling Alley,Thrift / Vintage Store,Beach,Food & Drink Shop,Design Studio,Eastern European Restaurant,Donut Shop,Diner
33,"(A subsidiary of Canara Bank)\r\n7th Floor, Na...",4,Café,Grocery Store,Bowling Alley,Thrift / Vintage Store,Beach,Food & Drink Shop,Design Studio,Eastern European Restaurant,Donut Shop,Diner
35,"The Hulkul\r# 81/37, Lavelle Road\rBangalore -...",4,Café,Grocery Store,Bowling Alley,Thrift / Vintage Store,Beach,Food & Drink Shop,Design Studio,Eastern European Restaurant,Donut Shop,Diner
42,"102, 'E' Wing, Mittal Towers, \r\nM. G. Road, ...",4,Café,Grocery Store,Bowling Alley,Thrift / Vintage Store,Beach,Food & Drink Shop,Design Studio,Eastern European Restaurant,Donut Shop,Diner
44,"102, 'E' Wing, Mittal Towers, \r\nM. G. Road, ...",4,Café,Grocery Store,Bowling Alley,Thrift / Vintage Store,Beach,Food & Drink Shop,Design Studio,Eastern European Restaurant,Donut Shop,Diner
56,"4th Floor, West Wing\r\n404, Raheja Towers\r\n...",4,Café,Grocery Store,Bowling Alley,Thrift / Vintage Store,Beach,Food & Drink Shop,Design Studio,Eastern European Restaurant,Donut Shop,Diner
65,"3rd Floor, 19/1, Infantry Road Cross\r\nBehind...",4,Café,Grocery Store,Bowling Alley,Thrift / Vintage Store,Beach,Food & Drink Shop,Design Studio,Eastern European Restaurant,Donut Shop,Diner
79,"iRely Solutions\r\nEmpire Infantry, \r\n3rd Fl...",4,Café,Grocery Store,Bowling Alley,Thrift / Vintage Store,Beach,Food & Drink Shop,Design Studio,Eastern European Restaurant,Donut Shop,Diner
100,"9th Floor, Du Parc Trinity, 17 M.G. Road, \nBa...",4,Café,Grocery Store,Bowling Alley,Thrift / Vintage Store,Beach,Food & Drink Shop,Design Studio,Eastern European Restaurant,Donut Shop,Diner


 <h2 align=center><font size = 4>Discussion</font></h2>

So we see that how we can apply the ML Algorithm K-Means clustering algorithm to help in finding the solution and analyze the result. This would not have been possible with the only available data. The clusters help anyone see the data and the common venues all the cluster have near them. Whether the cluster posses any possibility to open up a restaurant business where there are not enough options or no options. Of course this is not the only factor to be considered before taking the final call, but this is one of the method by which one can refer for the location wise analysis. As we can clearly see that Cluster 4 and Cluster 5 offer a great opportunity for restaurants to be opened as these places are less crowded. Also, Cluster 1, and 2 are not so great options for a new business to be step up. Cluster 3 also have a handful of option and but can be kept in the consideration as of any cuisine which is not currently available there. 